# Pré-processamento

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('df-final.csv')

In [4]:
df['acao'].unique()

KeyError: 'acao'

In [ ]:
previsores = df.drop(columns=['Unnamed: 0','poluicao_mundial', 'selo','prox_taxa', 'val_taxa', 'etapa_1', 'etapa_2'])
classe = df.iloc[:, 22].values #poluicao
previsores.head()

In [ ]:
#integração entre as bases de dados
df_resultado = df[(df['pessoa1'].isin(df_sent['Emp']) & df['pessoa2'].isin(df_sent['Agro']) & df['jogo'].isin(df_sent['Partida']) & df['rodada'].isin(df_sent['Rodada']) )].dropna().reset_index(drop=True)

In [ ]:
df_resultado

In [ ]:
#transformacao dos dados categoricos em numericos
previsores = previsores.drop(
    columns=['acao','pessoa1','pessoa2','objeto','tipo_preco','tratamento'])
previsores = pd.concat([previsores,pd.get_dummies(
    df[['acao','pessoa1','pessoa2','objeto','tipo_preco','tratamento']])], axis=1)
pd.set_option('display.max_columns', None)
previsores['parcela'].fillna(value=0, inplace=True)
previsores.head()

In [ ]:
#normalizacao dos dados
X = previsores
X = preprocessing.StandardScaler().fit(X).transform(X)
X

# Modelos

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
Y = classe
#separação dos dados para treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(X, Y, test_size = 0.3, random_state = 0)

Regressão Linear

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
regressaoLinear = LinearRegression()
regressaoLinear.fit(x_treino, y_treino)
score = regressaoLinear.score(x_treino, y_treino)
coeficiente = regressaoLinear.coef_
intercepcao = regressaoLinear.intercept_
previsoes = regressaoLinear.predict(x_teste)
mae = mean_absolute_error(y_teste, previsoes)
mse = mean_squared_error(y_teste, previsoes)

In [ ]:
print('Linear Regression')
print("Mean absolute error: " + str(mae))
print("Mean squared error: " + str(mse))
print("Score: " + str(score))

In [ ]:
y_pred = regressaoLinear.predict(x_teste)
df = pd.DataFrame({'Actual': y_teste, 'Predicted': y_pred})
df1 = df.head(30)
df1.plot(kind='bar',figsize=(10,8))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
regressaoArvores = DecisionTreeRegressor(random_state = 0)
regressaoArvores.fit(x_treino, y_treino)
score = regressaoArvores.score(x_treino, y_treino)
previsoes = regressaoArvores.predict(x_teste)
mae = mean_absolute_error(y_teste, previsoes)
mse = mean_squared_error(y_teste, previsoes)

In [ ]:
print('Decision Tree Regression')
print("Mean absolute error: " + str(mae))
print("Mean squared error: " + str(mse))
print("Score: " + str(score))

In [ ]:
y_pred = regressaoArvores.predict(x_teste)
df = pd.DataFrame({'Actual': y_teste, 'Predicted': y_pred})
df1 = df.head(30)
df1.plot(kind='bar',figsize=(10,8))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
regressaoRandomForest = RandomForestRegressor(n_estimators = 100)
regressaoRandomForest.fit(x_treino, y_treino)
score = regressaoRandomForest.score(x_treino, y_treino)
previsoes = regressaoRandomForest.predict(x_teste)
mae = mean_absolute_error(y_teste, previsoes)
mse = mean_squared_error(y_teste, previsoes)

In [ ]:
print('Random Forest Regression')
print("Mean absolute error: " + str(mae))
print("Mean squared error: " + str(mse))
print("Score: " + str(score))

In [ ]:
y_pred = regressaoRandomForest.predict(x_teste)
df = pd.DataFrame({'Actual': y_teste, 'Predicted': y_pred})
df1 = df.head(30)
df1.plot(kind='bar',figsize=(10,8))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

Support Vector Regression

In [ ]:
from sklearn.svm import SVR

In [ ]:
regressorSVM = SVR(kernel = 'rbf')
regressorSVM.fit(x_treino, y_treino)
score = regressorSVM.score(x_treino, y_treino)
previsoes = regressorSVM.predict(x_teste)
mae = mean_absolute_error(y_teste, previsoes)
mse = mean_squared_error(y_teste, previsoes)

In [ ]:
print('SVR Regression')
print("Mean absolute error: " + str(mae))
print("Mean squared error: " + str(mse))
print("Score: " + str(score))

In [ ]:
y_pred = regressorSVM.predict(x_teste)
df = pd.DataFrame({'Actual': y_teste, 'Predicted': y_pred})
df1 = df.head(30)
df1.plot(kind='bar',figsize=(10,8))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

# Plot Todas as Regressões

In [ ]:
params = {'legend.fontsize': 'large',          
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
plt.rcParams.update(params)

In [ ]:
fig, eixos = plt.subplots(nrows=2, ncols=2, figsize=(50,40))

eixos[0,0].plot(pd.DataFrame({'x': y_teste}).head(35), linewidth='2.5', color = '#72B5A4', label='Actual')
eixos[0,0].plot(pd.DataFrame({'y': regressaoLinear.predict(x_teste)}).head(35), linewidth='2.5', color = '#F15E5E', label='Predicted')
eixos[0,0].set_title('Linear Regression', fontsize=50)
eixos[0,0].legend(loc="upper right")
eixos[0,0].grid(which='major', linestyle='-', linewidth='0.5', color='green')
eixos[0,0].grid(which='minor', linestyle=':', linewidth='0.5', color='black')

eixos[0,1].plot(pd.DataFrame({'x': y_teste}).head(35), linewidth='2.5', color = '#72B5A4', label='Actual')
eixos[0,1].plot(pd.DataFrame({'y': regressorSVM.predict(x_teste)}).head(35), linewidth='2.5', color = '#F15E5E', label='Predicted')
eixos[0,1].set_title('SVM', fontsize=50)
eixos[0,1].legend(loc="upper right")
eixos[0,1].grid(which='major', linestyle='-', linewidth='0.5', color='green')
eixos[0,1].grid(which='minor', linestyle=':', linewidth='0.5', color='black')

eixos[1,0].plot(pd.DataFrame({'x': y_teste}).head(35), linewidth='2.5', color = '#72B5A4', label='Actual')
eixos[1,0].plot(pd.DataFrame({'y': regressaoArvores.predict(x_teste)}).head(35), linewidth='2.5', color = '#F15E5E', label='Predicted')
eixos[1,0].set_title('Decision Tree Regression', fontsize=50)
eixos[1,0].legend(loc="upper right")
eixos[1,0].grid(which='major', linestyle='-', linewidth='0.5', color='green')
eixos[1,0].grid(which='minor', linestyle=':', linewidth='0.5', color='black')

eixos[1,1].plot(pd.DataFrame({'x': y_teste}).head(35), linewidth='2.5', color = '#72B5A4', label='Actual')
eixos[1,1].plot(pd.DataFrame({'y': regressaoRandomForest.predict(x_teste)}).head(35), linewidth='2.5', color = '#F15E5E', label='Predicted')
eixos[1,1].set_title('Random Forest Regression', fontsize=50)
eixos[1,1].legend(loc="upper right")
eixos[1,1].grid(which='major', linestyle='-', linewidth='0.5', color='green')
eixos[1,1].grid(which='minor', linestyle=':', linewidth='0.5', color='black')

plt.subplots_adjust(wspace=0.2)
plt.subplots_adjust(hspace=0.3)
#plt.show()
plt.savefig('graphs.png')